In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Checking out the dataset

 we will be using the Newyork json file from the website 

In [8]:
with open('C:/Users/DR36495/Documents/MyGit/Capstone Project NY neighbourhood comparison Toronto/NYCJson.json')as json_data:
    ny_data=json.load(json_data)

In [10]:
ny_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [11]:
neighbourhood=ny_data['features']

In [13]:
neighbourhood[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [15]:
#creating dataframe for lat,lon and neighbourhood and boroughs
cols=['Borough', 'Neighborhood', 'Latitude', 'Longitude']

In [16]:
neigh=pd.DataFrame(columns=cols)

In [20]:
for data in neighbourhood:
    borough=data['properties']['borough']
    neigh_name=data['properties']['name']
    neigh_latlon=data['geometry']['coordinates']
    neigh_lat=neigh_latlon[0]
    neigh_lon=neigh_latlon[1]
    neigh=neigh.append({'Borough': borough,'Neighborhood': neigh_name,
'Latitude': neigh_lon,'Longitude': neigh_lat}, ignore_index=True)

In [21]:
neigh.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [35]:
print('The dataframe has {} boroughs and {} neighbourhood'.format(
    len(neigh['Borough'].unique()),neigh.shape[0]))


The dataframe has 5 boroughs and 306 neighbourhood


In [37]:
# Using Geolocator to plot
address='New York City, NY'
geolocator=Nominatim(user_agent='ny_explorer')
location=geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [45]:
#Create a  map of the city and add markers
map_newyork=folium.Map(location=[latitude,longitude],zoom_start=10)
#markers
for lat , lng, borough,neighbourhood in zip(neigh['Latitude'],neigh['Longitude'],neigh['Borough'],neigh['Neighborhood']):
    label='{},{}'.format(neighbourhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='purple',
    fill=True,
    fill_color='#7F00FF',
    fill_opacity=0.7,
    parse_html=False).add_to(map_newyork)
map_newyork

In [46]:
neigh.Borough.unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

In [123]:
#creating dataset for Brooklyn city
brooklyn_data=neigh[neigh['Borough']=='Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [48]:
address='Brooklyn ,NY'
geolocator=Nominatim(user_agent='ny_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The Geographic location of Brooklyn is {},{}'.format(latitude,longitude))

The Geographic location of Brooklyn is 40.6501038,-73.9495823


In [52]:
#creating a map
map_Brooklyn=folium.Map(location=[latitude,longitude],zoom_start=10)
#markers
for lat,lng,label in zip(brooklyn_data['Latitude'],brooklyn_data['Longitude']
                         ,brooklyn_data['Neighborhood']):
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_opacity=0.5,
    parse_html=False).add_to(map_Brooklyn)
map_Brooklyn

In [53]:
#Using FourSquare
CLIENT_ID='IUQMV40FTE1IMIAEOBV1SDMIFYTLV242NGKDWYVYKTCMGJVB'

CLIENT_SECRET='0EH4DMV3I5SY3M5GA51KZOCI4DBGGCQWIWY4IIUXMZY3PMV2'
VERSION='20190916'
LIMIT =100



In [61]:
neigh_lat=brooklyn_data.loc[0,'Latitude']
neigh_lng=brooklyn_data.loc[0,'Longitude']
neigh_name=brooklyn_data.loc[0,'Neighborhood']
print('Latitude and longitude values of {} are {}, {}.'.format(neigh_name, 
                                                               neigh_lat, 
                                                               neigh_lng))

Latitude and longitude values of Bay Ridge are 40.625801065010656, -74.03062069353813.


In [63]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_lng, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=IUQMV40FTE1IMIAEOBV1SDMIFYTLV242NGKDWYVYKTCMGJVB&client_secret=0EH4DMV3I5SY3M5GA51KZOCI4DBGGCQWIWY4IIUXMZY3PMV2&v=20190916&ll=40.625801065010656,-74.03062069353813&radius=500&limit=100'

In [113]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e2e122040a7ea001caa7e43'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Ridge',
  'headerFullLocation': 'Bay Ridge, Brooklyn',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 86,
  'suggestedBounds': {'ne': {'lat': 40.63030106951066,
    'lng': -74.02470273356597},
   'sw': {'lat': 40.62130106051065, 'lng': -74.03653865351028}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b895827f964a5206c2d32e3',
       'name': 'Pilo Arts Day Spa and Salon',
       'location': {'address': '8412 3rd Ave',
        'lat': 40.62474788273414,
        'lng': -74.03059056940135,
        'labeledLatL

In [114]:
venues=results['response']['groups'][0]['items']
print(venues)

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b895827f964a5206c2d32e3', 'name': 'Pilo Arts Day Spa and Salon', 'location': {'address': '8412 3rd Ave', 'lat': 40.62474788273414, 'lng': -74.03059056940135, 'labeledLatLngs': [{'label': 'display', 'lat': 40.62474788273414, 'lng': -74.03059056940135}], 'distance': 117, 'postalCode': '11209', 'cc': 'US', 'city': 'Brooklyn', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['8412 3rd Ave', 'Brooklyn, NY 11209', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d1ed941735', 'name': 'Spa', 'pluralName': 'Spas', 'shortName': 'Spa', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4b895827f964a5206c2d32e3-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reaso

In [115]:
nearby_venues=json_normalize(venues)#flatten JSON
print(nearby_venues)

                         referralId  reasons.count  \
0    e-0-4b895827f964a5206c2d32e3-0              0   
1    e-0-4ad09cf7f964a520bed820e3-1              0   
2    e-0-538ca7d5498ec684c6387a46-2              0   
3    e-0-4abe4e84f964a520958c20e3-3              0   
4    e-0-4ea17d24e5facb29c6d81d6f-4              0   
5    e-0-4bf5c0cebfeac92807269336-5              0   
6    e-0-5127e5c7e4b0447b9f67b41d-6              0   
7    e-0-4b6785d6f964a520aa532be3-7              0   
8    e-0-4bbce66da0a0c9b6b9f41a0f-8              0   
9    e-0-5089d4f7e4b0b2a614e157c5-9              0   
10  e-0-57bf2b24498e2a2800681ed3-10              0   
11  e-0-512967d1e4b01eb932845b7a-11              0   
12  e-0-4bd34524462cb7133601de07-12              0   
13  e-0-58fa85be98fbfc397bafe21f-13              0   
14  e-0-4b70d8dff964a52063312de3-14              0   
15  e-0-4a9880f5f964a520692c20e3-15              0   
16  e-0-4d92451d5cf5236ac70c4aa3-16              0   
17  e-0-4b19f066f964a52062e6

In [118]:
#filter cols
filtered_cols=['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_cols]
print(nearby_venues['venue.categories'].head())

0    [{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...
1    [{'id': '4bf58dd8d48988d179941735', 'name': 'B...
2    [{'id': '4bf58dd8d48988d112941735', 'name': 'J...
3    [{'id': '4bf58dd8d48988d143941735', 'name': 'B...
4    [{'id': '4bf58dd8d48988d151941735', 'name': 'T...
Name: venue.categories, dtype: object


In [119]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
               
               
    

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


In [120]:
#clean cols

nearby_venues.columns=[col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Pilo Arts Day Spa and Salon,Spa,40.624748,-74.030591
1,Bagel Boy,Bagel Shop,40.627896,-74.029335
2,Cocoa Grinder,Juice Bar,40.623967,-74.030863
3,Pegasus Cafe,Breakfast Spot,40.623168,-74.031186
4,Ho' Brah Taco Joint,Taco Place,40.622960,-74.031371


In [121]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))


86 venues were returned by Foursquare.


In [125]:
def getNearbyVenues(names,latitudes,longitudes,radius=500):
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
        #create API request for URL
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
        

In [126]:
Brooklyn_venues=getNearbyVenues(names=brooklyn_data['Neighborhood'],
                               latitudes=brooklyn_data['Latitude'],
                               longitudes=brooklyn_data['Longitude'])

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [145]:
brooklyn_onehot['Neighborhood']=Brooklyn_venues['Neighborhood']
brooklyn_onehot['Neighborhood']

0                       Bay Ridge
1                       Bay Ridge
2                       Bay Ridge
3                       Bay Ridge
4                       Bay Ridge
5                       Bay Ridge
6                       Bay Ridge
7                       Bay Ridge
8                       Bay Ridge
9                       Bay Ridge
10                      Bay Ridge
11                      Bay Ridge
12                      Bay Ridge
13                      Bay Ridge
14                      Bay Ridge
15                      Bay Ridge
16                      Bay Ridge
17                      Bay Ridge
18                      Bay Ridge
19                      Bay Ridge
20                      Bay Ridge
21                      Bay Ridge
22                      Bay Ridge
23                      Bay Ridge
24                      Bay Ridge
25                      Bay Ridge
26                      Bay Ridge
27                      Bay Ridge
28                      Bay Ridge
29            

In [196]:
#one hot encoding
brooklyn_onehot=pd.get_dummies(Brooklyn_venues[['Venue Category']],prefix="",prefix_sep="")
#add neighborhood cols to df
brooklyn_onehot['Neighborhood']=Brooklyn_venues['Neighborhood']

#move neighborhood to first col
fixed_columns=[brooklyn_onehot.columns[-1]]+list(brooklyn_onehot.columns[:-1])
brooklyn_onehot=brooklyn_onehot[fixed_columns]
#brooklyn_onehot.drop(brooklyn_onehot.columns[181],axis=1,inplace=True)
brooklyn_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Resta

In [198]:
brooklyn_grouped=brooklyn_onehot.groupby("Neighborhood").mean().reset_index()

In [205]:
num_top_venues=5
for hood in brooklyn_grouped['Neighborhood']:
    print('***'+hood+'***')
    var=brooklyn_grouped[brooklyn_grouped['Neighborhood']==hood].T.reset_index()
    var.columns=['venue','freq']
    var=var.iloc[1:]
    var['freq']=var['freq'].astype(float)
    var=var.round({'freq':2})
    print(var.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

***Bath Beach***
                  venue  freq
0    Chinese Restaurant  0.08
1           Pizza Place  0.06
2              Pharmacy  0.06
3  Fast Food Restaurant  0.04
4      Sushi Restaurant  0.04


***Bay Ridge***
                venue  freq
0  Italian Restaurant  0.08
1                 Spa  0.06
2            Pharmacy  0.03
3                 Bar  0.03
4          Bagel Shop  0.03


***Bedford Stuyvesant***
         venue  freq
0         Café  0.08
1  Coffee Shop  0.08
2          Bar  0.08
3  Pizza Place  0.08
4    BBQ Joint  0.04


***Bensonhurst***
                venue  freq
0          Donut Shop  0.07
1      Ice Cream Shop  0.07
2  Chinese Restaurant  0.07
3  Italian Restaurant  0.07
4    Sushi Restaurant  0.07


***Bergen Beach***
               venue  freq
0    Harbor / Marina  0.25
1     Baseball Field  0.12
2         Donut Shop  0.12
3               Park  0.12
4  Other Repair Shop  0.12


***Boerum Hill***
                    venue  freq
0            Dance Studio  0.06
1        

                 venue  freq
0          Pizza Place   0.1
1             Pharmacy   0.1
2   Spanish Restaurant   0.1
3  Fried Chicken Joint   0.1
4   Chinese Restaurant   0.1


***North Side***
                 venue  freq
0          Coffee Shop  0.07
1             Wine Bar  0.04
2                  Bar  0.04
3          Yoga Studio  0.03
4  American Restaurant  0.03


***Ocean Hill***
                             venue  freq
0                    Deli / Bodega  0.21
1                         Bus Stop  0.10
2  Southern / Soul Food Restaurant  0.07
3              Fried Chicken Joint  0.07
4                    Grocery Store  0.07


***Ocean Parkway***
              venue  freq
0       Pizza Place  0.08
1      Liquor Store  0.08
2       Men's Store  0.04
3  Sushi Restaurant  0.04
4         Gift Shop  0.04


***Paerdegat Basin***
                venue  freq
0     Harbor / Marina  0.14
1         Auto Garage  0.14
2  Child Care Service  0.14
3       Deli / Bodega  0.14
4  Photography Studio  0.1

In [206]:
def return_most_common_venues(row,num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [214]:
num_top_venues=10
indicators=['st','nd','rd']
#create cols accordin to num of top venues
cols=['Neighborhood']
print(cols)
for ind in np.arange(num_top_venues):
    try:
        cols.append('{}{} most common venue'.format(ind+1,indicators[ind]))
    except:
        cols.append('{}th most common venue'.format(ind+1))
#create a new df
neigh_venues_sorted=pd.DataFrame(columns=cols)
neigh_venues_sorted['Neighborhood']=brooklyn_grouped['Neighborhood']
for ind in np.arange(brooklyn_grouped.shape[0]):
    neigh_venues_sorted.iloc[ind,1:]=return_most_common_venues(brooklyn_grouped.iloc[ind,:],num_top_venues)
neigh_venues_sorted.head()
    
    
        

['Neighborhood']


,Neighborhood,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,Bath Beach,Chinese Restaurant,Pharmacy,Pizza Place,Donut Shop,Bubble Tea Shop,Italian Restaurant,Fast Food Restaurant,Asian Restaurant,Sushi Restaurant,Dessert Shop
1,Bay Ridge,Italian Restaurant,Spa,American Restaurant,Bagel Shop,Bar,Greek Restaurant,Pizza Place,Pharmacy,Sushi Restaurant,Grocery Store
2,Bedford Stuyvesant,Coffee Shop,Café,Pizza Place,Bar,BBQ Joint,Fried Chicken Joint,Fruit & Vegetable Store,Boutique,New American Restaurant,Gourmet Shop
3,Bensonhurst,Sushi Restaurant,Chinese Restaurant,Ice Cream Shop,Donut Shop,Italian Restaurant,Pharmacy,Supermarket,Noodle House,Cha Chaan Teng,Record Shop
4,Bergen Beach,Harbor / Marina,Athletics & Sports,Donut Shop,Baseball Field,Park,Other Repair Shop,Playground,Flower Shop,Flea Market,Fish Market


In [216]:
brooklyn_grouped.iloc[0,:]

Neighborhood                                Bath Beach
Yoga Studio                                          0
Accessories Store                                    0
American Restaurant                                  0
Antique Shop                                         0
Arepa Restaurant                                     0
Argentinian Restaurant                               0
Art Gallery                                          0
Arts & Crafts Store                                  0
Arts & Entertainment                                 0
Asian Restaurant                             0.0377358
Athletics & Sports                                   0
Auto Dealership                                      0
Auto Garage                                          0
BBQ Joint                                            0
Bagel Shop                                   0.0188679
Bakery                                       0.0188679
Bank                                         0.0188679
Bar       

In [217]:
brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape Store,Varenyky restaurant,Vegetarian / Vegan Resta

In [218]:
#set of clusters 
kcluster=5
brooklyn_grouped_clustering=brooklyn_grouped.drop('Neighborhood',1)

#run kmeans clustering
kmeans=KMeans(n_clusters=kcluster,random_state=0).fit(brooklyn_grouped_clustering)
 #check labels generated for each cluster for each row from df
kmeans.labels_[0:10]

array([4, 0, 0, 4, 0, 0, 4, 0, 0, 0])

In [219]:
neigh_venues_sorted

,Neighborhood,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,Bath Beach,Chinese Restaurant,Pharmacy,Pizza Place,Donut Shop,Bubble Tea Shop,Italian Restaurant,Fast Food Restaurant,Asian Restaurant,Sushi Restaurant,Dessert Shop
1,Bay Ridge,Italian Restaurant,Spa,American Restaurant,Bagel Shop,Bar,Greek Restaurant,Pizza Place,Pharmacy,Sushi Restaurant,Grocery Store
2,Bedford Stuyvesant,Coffee Shop,Café,Pizza Place,Bar,BBQ Joint,Fried Chicken Joint,Fruit & Vegetable Store,Boutique,New American Restaurant,Gourmet Shop
3,Bensonhurst,Sushi Restaurant,Chinese Restaurant,Ice Cream Shop,Donut Shop,Italian Restaurant,Pharmacy,Supermarket,Noodle House,Cha Chaan Teng,Record Shop
4,Bergen Beach,Harbor / Marina,Athletics & Sports,Donut Shop,Baseball Field,Park,Other Repair Shop,Playground,Flower Shop,Flea Market,Fish Market
5,Boerum Hill,Dance Studio,Bar,Coffee Shop,Furniture / Home Store,French Restaurant,Arts & Crafts Store,Sandwich Place,Kids Store,Grocery Store,Gym / Fitness Center
6,Borough Park,Bank,Pizza Place,Pharmacy,Hotel,American Restaurant,Deli / Bodega,Fast Food Restaurant,Bistro,Café,Restaurant
7,Brighton Beach,Beach,Russian Restaurant,Restaurant,Eastern European Restaurant,Bank,Gourmet Shop,Sushi Restaurant,Mobile Phone Shop,Pharmacy,Lounge
8,Broadway Junction,Diner,Donut Shop,Ice Cream Shop,Grocery Store,Sandwich Place,Caribbean Restaurant,Burger Joint,Moving Target,Metro Station,Gas Station
9,Brooklyn Heights,Yoga Studio,Park,Deli / Bodega,Cosmetics Shop,Italian Restaurant,Gym,Bakery,Coffee Shop,Bar,Diner


In [236]:
#add clustering labels
neigh_venues_sorted.insert(0,'Cluster Labels',kmeans.labels_)
brooklyn_merged=brooklyn_data
#merge brooklyn_grouped with brooklyn_data to get lat and lng for each neighborhood
brooklyn_merged=brooklyn_data.join(neigh_venues_sorted.set_index('Neighborhood'),on
                                  ='Neighborhood')


ValueError: cannot insert Cluster Labels, already exists

In [239]:
brooklyn_merged['Cluster Labels'].value_counts()#check the last cols

0    38
4    26
2     4
3     1
1     1
Name: Cluster Labels, dtype: int64

In [240]:
# create map
map_cluster=folium.Map(location=[latitude,longitude],zoom_start=10)
#set color for cluster
x=np.arange(kcluster)
ys=[i+x+(i*x)**2 for i in range(kcluster)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i)for i in colors_array]
#add markers
markers_colors=[]
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'],brooklyn_merged['Longitude'],
                                  brooklyn_merged['Neighborhood'],brooklyn_merged['Cluster Labels']):
                                  label=folium.Popup(brooklyn_merged['Cluster Labels'],parse_html=True)
                                  folium.CircleMarker(
                                  [lat,lng],
                                  radius=6,
                                popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster)
map_cluster